In [19]:
import pandas as pd

In [20]:
df = pd.read_csv('resume_data.csv')

In [21]:
print(df.columns)

Index(['address', 'career_objective', 'skills', 'educational_institution_name',
       'degree_names', 'passing_years', 'educational_results', 'result_types',
       'major_field_of_studies', 'professional_company_names', 'company_urls',
       'start_dates', 'end_dates', 'related_skils_in_job', 'positions',
       'locations', 'responsibilities', 'extra_curricular_activity_types',
       'extra_curricular_organization_names',
       'extra_curricular_organization_links', 'role_positions', 'languages',
       'proficiency_levels', 'certification_providers', 'certification_skills',
       'online_links', 'issue_dates', 'expiry_dates', '﻿job_position_name',
       'educationaL_requirements', 'experiencere_requirement',
       'age_requirement', 'responsibilities.1', 'skills_required',
       'matched_score'],
      dtype='object')


UNPACKING PART

In [22]:
#Data types of columns
categorizedColumns = ['skills', 'educational_institution_name', 'degree_names', 'major_field_of_studies', 'professional_company_names','related_skils_in_job', 'responsibilities', 'extra_curricular_organization_names','extra_curricular_activity_types','certification_skills','responsibilities.1','skills_required']
essayColumns = ['career_objective', 'educationaL_requirements']
numericalColumns = ['matched_score']
dateColumns = ['passing_years','start_dates', 'end_dates','issue_dates', 'expiry_dates','experiencere_requirement']
ageColumns = ['age_requirement']
#Categorize columns
individualInfo = ['address', 'career_objective', 'skills']
educationInfo = ['educational_institution_name','degree_names', 'passing_years', 'educational_results', 'result_types','major_field_of_studies']
professionalInfo = ['professional_company_names', 'company_urls','start_dates', 'end_dates', 'related_skils_in_job', 'positions','locations', 'responsibilities']
extraCurricularInfo = ['extra_curricular_activity_types','extra_curricular_organization_names','extra_curricular_organization_links', 'role_positions']
languageInfo = ['language_names', 'language_proficiency_levels']
certificationInfo = ['certification_names', 'certification_skills', 'issue_dates', 'expiry_dates']
jobInfo = ['job_titles', 'job_company_names', 'job_locations', 'job_start_dates', 'job_end_dates', 'responsibilities.1', 'skills_required', 'educationaL_requirements']
similarity = ['matched_score']
jobTitleComparison = ['positions', 'role_positions', 'job_titles']
responsibilityComparison = ['responsibilities', 'responsibilities.1']
educationComparison = ['degree_names', 'major_field_of_studies', 'educationaL_requirements']
skillsComparison = ['skills', 'related_skils_in_job', 'skills_required']
skillsCertificationComparison = ['skills', 'related_skils_in_job', 'certification_skills']

In [23]:
import ast

#Regex motherfucker
# Convert the 'skills' column from string representation of list to actual list
df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [24]:
from collections import Counter

# Ensure all entries in 'skills' column are lists
df['skills'] = df['skills'].apply(lambda x: x if isinstance(x, list) else [])

# Flatten the list of skills and count the occurrences
skills_counter = Counter(skill for sublist in df['skills'] for skill in sublist)

# Convert the counter to a DataFrame for better readability
skills_df = pd.DataFrame.from_dict(skills_counter, orient='index', columns=['Count']).reset_index()
skills_df.rename(columns={'index': 'Skill'}, inplace=True)

print(skills_df)

                   Skill  Count
0               Big Data    168
1                 Hadoop    140
2                   Hive    168
3                 Python   3612
4              Mapreduce     28
...                  ...    ...
3338        Communicator     28
3339       quick learner     28
3340       Data Handling     28
3341  Customer Analytics     28
3342                  MS     28

[3343 rows x 2 columns]


In [25]:
# Convert the text columns into arrays by splitting on newlines and removing empty strings
df['skills_required'] = df['skills_required'].str.split('\n').apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else [])
df['responsibilities.1'] = df['responsibilities.1'].str.split('\n').apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else [])

In [26]:
# Get the initial count of rows
initial_count = len(df)

# Get all columns except matched_score
cols_to_check = [col for col in df.columns if col != 'matched_score']

# Remove rows where all columns (except matched_score) are empty/null
df_cleaned = df.dropna(subset=cols_to_check, how='all')

# Calculate number of rows removed
rows_removed = initial_count - len(df_cleaned)

print(f"Number of rows removed: {rows_removed}")

Number of rows removed: 0


In [27]:
# Convert all null and missing values to None
df = df.where(pd.notnull(df), None)

In [28]:
# Create a function to count elements in array-like strings
def count_elements(x):
    if isinstance(x, list):
        return len(x)
    elif isinstance(x, str):
        try:
            # Try to evaluate string as list
            lst = ast.literal_eval(x)
            return len(lst)
        except:
            # If string can't be evaluated as list, split on newlines
            return len(x.split('\n'))
    return 0

# Add count columns for categorized columns
for col in categorizedColumns:
    new_col_name = f'{col}_count'
    df[new_col_name] = df[col].apply(count_elements)

In [29]:
import nltk
from nltk.corpus import wordnet as wn
from itertools import product

# Ensure you have the WordNet data
nltk.download('wordnet')
nltk.download('punkt')

def sentence_similarity(sentence1, sentence2):
    # Tokenize sentences
    words1 = nltk.word_tokenize(sentence1)
    words2 = nltk.word_tokenize(sentence2)
    
    # Get synsets for each word in the sentences
    synsets1 = [wn.synsets(word) for word in words1]
    synsets2 = [wn.synsets(word) for word in words2]
    
    # Filter out empty synsets
    synsets1 = [synset[0] for synset in synsets1 if synset]
    synsets2 = [synset[0] for synset in synsets2 if synset]
    
    # Compute similarity between all synset pairs
    similarities = []
    for synset1, synset2 in product(synsets1, synsets2):
        similarity = synset1.path_similarity(synset2)
        if similarity is not None:
            similarities.append(similarity)
    
    # Average the similarities
    if similarities:
        return sum(similarities) / len(similarities)
    else:
        return 0
    
# Example sentences
sentence1 = "Technical Support Troubleshooting Collaboration"
sentence2 = "Machine Learning Leadership Cross-Functional Collaboration"

# Compute sentence similarity
similarity = sentence_similarity(sentence1, sentence2)
print(f"Sentence similarity: {similarity}")

Sentence similarity: 0.16806414072039072


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
from sentence_transformers import SentenceTransformer, util

def compute_sentence_similarity(sentence1, sentence2):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # Compute sentence embeddings
    embeddings = model.encode([sentence1, sentence2])

    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    return similarity.item()


In [31]:
def pathSimilarity(firstSentence, secondSentence, seperator = r'[\n.]'):
    firstSentence = firstSentence.split(seperator)
    secondSentence = secondSentence.split(seperator)
    similarity = []
    setSimilarity = []
    for i in firstSentence:
        for j in secondSentence:
            setSimilarity.append(compute_sentence_similarity(i,j))
        similarity.append(max(setSimilarity))
        setSimilarity = []
    return sum(similarity)/len(similarity)

def cosineSimilarity(firstSentence, secondSentence, seperator = r'[\n.]'):
    firstSentence = firstSentence.split(seperator)
    secondSentence = secondSentence.split(seperator)
    similarity = []
    setSimilarity = []
    for i in firstSentence:
        for j in secondSentence:
            setSimilarity.append(sentence_similarity(i,j))
        similarity.append(max(setSimilarity))
        setSimilarity = []
    return sum(similarity)/len(similarity)
    

In [32]:
# Count skills in skills_required column and create new feature
df['skills_required_count'] = df['skills_required'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['skills_count'] = df['skills'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Print first few rows to verify
print("First 5 rows of skills_required_count:")
print(df['skills_required_count'].head())
print("\nFirst 5 rows of skills_count:")
print(df['skills_count'].head())    

First 5 rows of skills_required_count:
0    0
1    0
2    6
3    2
4    8
Name: skills_required_count, dtype: int64

First 5 rows of skills_count:
0    21
1    10
2    14
3    36
4    32
Name: skills_count, dtype: int64


In [34]:
# Create similarity score for each row by comparing skills to certification_skills
df['skills_certification_path_similarity'] = df.apply(lambda row: 
    pathSimilarity(
        ' '.join(row['skills']) if isinstance(row['skills'], list) else '',
        ' '.join(row['certification_skills']) if isinstance(row['certification_skills'], list) else ''
    ) if row['skills'] is not None and row['certification_skills'] is not None else 0, 
    axis=1
)

# Create similarity score for each row by comparing skills to certification_skills
df['skills_certification_cosine_similarity'] = df.apply(lambda row: 
    cosineSimilarity(
        ' '.join(row['skills']) if isinstance(row['skills'], list) else '',
        ' '.join(row['certification_skills']) if isinstance(row['certification_skills'], list) else ''
    ) if row['skills'] is not None and row['certification_skills'] is not None else 0, 
    axis=1
)

# Print first few rows to verify
print("First 5 rows of skills_certification_similarity:")
print(df['skills_certification_cosine_similarity'].head())

First 5 rows of skills_certification_similarity:
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: skills_certification_cosine_similarity, dtype: float64


In [ ]:
# Create similarity score for each row by comparing skills to certification_skills
df['skills_certification_path_similarity'] = df.apply(lambda row: 
    pathSimilarity(
        ' '.join(row['skills_related']) if isinstance(row['skills'], list) else '',
        ' '.join(row['certification_skills']) if isinstance(row['certification_skills'], list) else ''
    ) if row['skills'] is not None and row['certification_skills'] is not None else 0, 
    axis=1
)

# Create similarity score for each row by comparing skills to certification_skills
df['skills_certification_cosine_similarity'] = df.apply(lambda row: 
    cosineSimilarity(
        ' '.join(row['skills_related']) if isinstance(row['skills'], list) else '',
        ' '.join(row['certification_skills']) if isinstance(row['certification_skills'], list) else ''
    ) if row['skills'] is not None and row['certification_skills'] is not None else 0, 
    axis=1
)

# Print first few rows to verify
print("First 5 rows of skills_certification_similarity:")
print(df['skills_certification_similarity'].head())